In [251]:
#Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import sem


In [232]:
#Study data files
#import files into variables and read the csv files
mouse_metadata=pd.read_csv("data/Mouse_metadata.csv")
mouse_metadata

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16
...,...,...,...,...,...
244,z314,Stelasyn,Female,21,28
245,z435,Propriva,Female,12,26
246,z581,Infubinol,Female,24,25
247,z795,Naftisol,Female,13,29


In [233]:
study_results=pd.read_csv("data/Study_results.csv")
study_results

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.000000,0
1,f932,0,45.000000,0
2,g107,0,45.000000,0
3,a457,0,45.000000,0
4,c819,0,45.000000,0
...,...,...,...,...
1888,r944,45,41.581521,2
1889,u364,45,31.023923,3
1890,p438,45,61.433892,1
1891,x773,45,58.634971,4


In [234]:
#combine datas into 1 set
mouse_study= pd.merge(mouse_metadata,study_results,on="Mouse ID", how="left")
mouse_study

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [235]:
mouse_study=mouse_study.rename(columns={"Mouse ID":"Mouse_id","Drug Regimen":"Drug_regimen","Weight (g)":"Weight","Tumor Volume (mm3)":"Tumor_volume","Metastatic Sites":"Metastatic_sites"})
mouse_study.head()

,Mouse_id,Drug_regimen,Sex,Age_months,Weight,Timepoint,Tumor_volume,Metastatic_sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [236]:
#Check the number of mice
mouse_study['Mouse_id'].count()
mouse_study.count()

Mouse_id            1893
Drug_regimen        1893
Sex                 1893
Age_months          1893
Weight              1893
Timepoint           1893
Tumor_volume        1893
Metastatic_sites    1893
dtype: int64

In [228]:
#Other option to duplicate datas- datas for duplicate mouse id
mouse_groupby=mouse_study.groupby(['Mouse_id', 'Timepoint'])["Mouse_id"].unique()
mouse_study=mouse_study[mouse_study.duplicated(subset=['Mouse_id','Timepoint'],keep=False)]
print(mouse_study)

Empty DataFrame
Columns: [Mouse_id, Drug_regimen, Sex, Age_months, Weight, Timepoint, Tumor_volume, Metastatic_sites]
Index: []


In [237]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_study.sort_values("Mouse_id",inplace=True)
mouse_study.drop_duplicates(subset=["Mouse_id","Timepoint"], keep=False, inplace=True)



In [238]:
#checking the number of mice is clean data
mouse_study.count()

Mouse_id            1883
Drug_regimen        1883
Sex                 1883
Age_months          1883
Weight              1883
Timepoint           1883
Tumor_volume        1883
Metastatic_sites    1883
dtype: int64

In [240]:
#Summary Statistics
#Mean data for tumor volume for each regimen
mean_tumor_volume= mouse_study.groupby(["Drug_regimen"])["Tumor_volume"].mean()
mean_tumor_volume

Drug_regimen
Capomulin    40.675741
Ceftamin     52.591172
Infubinol    52.884795
Ketapril     55.235638
Naftisol     54.331565
Placebo      54.033581
Propriva     52.458254
Ramicane     40.216745
Stelasyn     54.233149
Zoniferol    53.236507
Name: Tumor_volume, dtype: float64

In [241]:
#Median data for tumor volume for each regimen
median_tumor_volume= mouse_study.groupby(["Drug_regimen"])["Tumor_volume"].median()
median_tumor_volume

Drug_regimen
Capomulin    41.557809
Ceftamin     51.776157
Infubinol    51.820584
Ketapril     53.698743
Naftisol     52.509285
Placebo      52.288934
Propriva     50.854632
Ramicane     40.673236
Stelasyn     52.431737
Zoniferol    51.818479
Name: Tumor_volume, dtype: float64

In [242]:
#Variance data for tumor volume for each regimen
var_tumor_volume= mouse_study.groupby(["Drug_regimen"])["Tumor_volume"].var()
var_tumor_volume

Drug_regimen
Capomulin    24.947764
Ceftamin     39.290177
Infubinol    43.128684
Ketapril     68.553577
Naftisol     66.173479
Placebo      61.168083
Propriva     44.053659
Ramicane     23.486704
Stelasyn     59.450562
Zoniferol    48.533355
Name: Tumor_volume, dtype: float64

In [243]:
#Standard deviation for tumor volume for each regimen
std_tumor_volume= mouse_study.groupby(["Drug_regimen"])["Tumor_volume"].std()
std_tumor_volume

Drug_regimen
Capomulin    4.994774
Ceftamin     6.268188
Infubinol    6.567243
Ketapril     8.279709
Naftisol     8.134708
Placebo      7.821003
Propriva     6.637293
Ramicane     4.846308
Stelasyn     7.710419
Zoniferol    6.966589
Name: Tumor_volume, dtype: float64

In [262]:
sample=mouse_study.sample(100)
mouse_study.sem(axis=0, skipna=False)



Age_months          0.165550
Weight              0.090611
Timepoint           0.324704
Tumor_volume        0.205437
Metastatic_sites    0.026255
dtype: float64